In [1]:
import pandas as pd

In [2]:
PPG_dict = {
    'PPG-CCMC': '55134',
    'PPG-Mat': '55135',
    'PPG-PROFMAT': '55136',
    'PPG-MECAI': '55137',
    'PPG-PIPGEs': '104131'
}

def cria_link( codigo, curso ):

    codigo = codigo.strip()
    curso = curso.strip()

    try:
        ppg = PPG_dict[ curso ]
        return f'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa={ppg}&disciplina={codigo}'
    except KeyError:
        return f'https://uspdigital.usp.br/jupiterweb/obterDisciplina?nomdis=&sgldis={codigo}'


In [3]:
df = pd.read_excel('Elenco SME_2025-2-ICMC_corrigido.xlsx', skiprows = 2)
# df = pd.read_csv('../../Elenco SME_2025-2-ICMC.csv', delimiter=';')

In [4]:
df = df.dropna(subset=["Disciplina (código)"])
df = df.fillna( '' )

In [5]:
df = df.rename(columns={"Disciplina (código)": "codigo", 
                        "Disciplina (nome completo)": "nome", 
                        "Curso(s)": "curso",
                        "Horário 1": "horario1", "Horário 2": "horario2", "Turma": "turma"})
df = df.drop( [ 'Horário 3', 'Horário 4', 'observações', 'Docente\n(nome completo sem abreviações)', 'NUSP', 'Será espelho com Pós?', 'Utilizará laboratório?\n(sim ou não)', 'Sala \n(a definir)', 'créditos' ], axis = 1 )

In [6]:
ementa = []
nivel = []
for codigo, curso in zip( df[ 'codigo' ], df[ 'curso' ] ):
    ementa.append( cria_link( codigo, curso ) )
    try:
        p = PPG_dict[ curso ]
        nivel.append( 'p' )
    except KeyError:
        nivel.append( 'g' )

df[ 'ementa' ] = ementa
df[ 'nivel' ] = nivel

In [7]:
df = df.astype({'turma': 'int32'})

In [8]:
df['carga'] = df['carga'].astype('float16')

In [9]:
df

,codigo,nome,curso,turma,horario1,horario2,grupo,carga,ementa,nivel
0,SME0142,Algebra Linear e Aplicações,"BCC, BCDados",1,Terça - 10:10 / 11:50,Quinta - 10:10 / 11:50,Matemática,1.0,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
1,SME0142,Algebra Linear e Aplicações,BCC,2,Terça - 08:10 / 09:50,Quinta - 08:10 / 09:50,Matemática,1.0,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
2,SME0123,Estatística,BCC,1,Terça - 14:20 / 16:00,Sexta - 08:10 / 09:50,Estatística,1.0,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
3,SME0123,Estatística,BCC,2,Terça - 16:20 / 18:00,Sexta - 10:10 / 11:50,Estatística,1.0,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
4,SME0110,Programação Matemática,BCC,1,Segunda - 08:10 / 09:50,Quarta - 10:10 / 11:50,Otimização,1.0,https://uspdigital.usp.br/jupiterweb/obterDisc...,g
...,...,...,...,...,...,...,...,...,...,...
66,EST5106,Tópicos de pesquisa II,PPG-PIPGEs,1,a definir,,Estatística,1.0,https://www.icmc.usp.br/pos-graduacao/discipli...,p
67,EST5519,Capacitação Docente em Estatística I,PPG-PIPGEs,1,a definir,,Estatística,0.5,https://www.icmc.usp.br/pos-graduacao/discipli...,p
68,EST5521,Capacitação Docente em Estatística II,PPG-PIPGEs,1,a definir,,Estatística,0.5,https://www.icmc.usp.br/pos-graduacao/discipli...,p
69,EST5804,Tópicos Avançados de Pesquisa I,PPG-PIPGEs,1,a definir,,Estatística,1.0,https://www.icmc.usp.br/pos-graduacao/discipli...,p


In [10]:
json_data = df.to_dict(orient='records')

In [11]:
for item in json_data:
    keys_to_remove = [k for k, v in item.items() if pd.isna(v) and k == "horario2"]
    for k in keys_to_remove:
        del item[k]

In [12]:
len(json_data)

71

In [13]:
dias_map = {
    "Segunda": "Seg.",
    "Terça": "Ter.",
    "Quarta": "Qua.",
    "Quinta": "Qui.",
    "Sexta": "Sex.",
    "Sábado": "Sáb.",
    "Domingo": "Dom."
}

In [14]:
def processar_horario(horario):
    if pd.isna(horario):
        return None
    try:
        # Exemplo: "Terça - 14:20 / 16:00"
        dia_e_horas = horario.split('-')
        dia_extenso = dia_e_horas[0].strip()
        dia = dias_map.get(dia_extenso, dia_extenso)  # Se não tiver no mapa, mantém original
        horas = dia_e_horas[1].strip().split('/')
        inicio = horas[0].strip()
        fim = horas[1].strip()
        return {"dia": dia, "inicio": inicio, "fim": fim}
    except Exception as e:
        print(f"Erro ao processar: {horario} -> {e}")
        return None

In [15]:
for item in json_data:
    horarios = []
    for key in ['horario1', 'horario2']:
        if key in item:
            h = processar_horario(item[key])
            if h:
                horarios.append(h)
            del item[key]  # Remove os campos antigos

    # if horarios:
    item['horarios'] = horarios
    item['id'] = f"{item['codigo'].rstrip().lstrip()},{item['turma']}"


Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar:  -> list index out of range
Erro ao processar: a definir  -> list index out of range
Erro ao processar

In [16]:
len(json_data)

71

In [17]:
from datetime import datetime

In [18]:
def hora_para_minutos(hora_str):
    """Converte 'HH:MM' para minutos desde 00:00"""
    h, m = map(int, hora_str.split(":"))
    return h * 60 + m

In [19]:
def horarios_conflitam(h1, h2):
    """Verifica se dois horários conflitam no mesmo dia"""
    if h1['dia'] != h2['dia']:
        return False

    inicio1 = hora_para_minutos(h1['inicio'])
    fim1 = hora_para_minutos(h1['fim'])
    inicio2 = hora_para_minutos(h2['inicio'])
    fim2 = hora_para_minutos(h2['fim'])

    return max(inicio1, inicio2) < min(fim1, fim2)

In [20]:
# 16:00 -> 18:00
a = {"inicio": "16:00", "fim": "18:00", "dia": "Seg."}
# 14:00 -> 16:00
b = {"inicio": "14:00", "fim": "16:01", "dia": "Seg."}

# 16:00 -> 18:00
a = {"inicio": "16:00", "fim": "18:00", "dia": "Seg."}
# 14:00 -> 16:00
b = {"inicio": "18:00", "fim": "19:01", "dia": "Seg."}

horarios_conflitam(a, b)

False

In [21]:
# Inicializa os sets de conflitos
for turma in json_data:
    turma['conflitos'] = set()

# Compara todas as turmas em pares (sem repetições)
for i in range(len(json_data)):
    turma_i = json_data[i]
    for j in range(i + 1, len(json_data)):
        turma_j = json_data[j]

        # Verifica se algum horário da turma_i conflita com algum da turma_j
        for h1 in turma_i.get('horarios', []):
            for h2 in turma_j.get('horarios', []):
                if horarios_conflitam(h1, h2):
                    turma_i['conflitos'].add(turma_j['id'])
                    turma_j['conflitos'].add(turma_i['id'])
                    break  # Um conflito já basta

# Converte os sets para listas (JSON serializável)
for turma in json_data:
    turma['conflitos'] = list(turma['conflitos'])

In [22]:
def marcar_turmas_noturnas(json_data):
    for turma in json_data:
        turma['noturna'] = False  # valor padrão
        for horario in turma.get('horarios', []):
            try:
                hora, minuto = map(int, horario['inicio'].split(':'))
                if hora >= 18:
                    turma['noturna'] = True
                    break  # Já é noturna, não precisa verificar os outros horários
            except Exception as e:
                print(f"Erro ao processar horário {horario['inicio']} da turma {turma.get('id')}: {e}")


In [23]:
marcar_turmas_noturnas(json_data)

In [24]:
json_data.sort(key=lambda x: x['codigo'])


In [25]:
import json

In [26]:
with open(r'turmas.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

In [27]:
json_data

[{'codigo': 'EST5102',
  'nome': 'Inferência Estatística',
  'curso': 'PPG-PIPGEs',
  'turma': 1,
  'grupo': 'Estatística',
  'carga': 1.0,
  'ementa': 'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa=104131&disciplina=EST5102',
  'nivel': 'p',
  'horarios': [],
  'id': 'EST5102,1',
  'conflitos': [],
  'noturna': False},
 {'codigo': 'EST5106',
  'nome': 'Tópicos de pesquisa II ',
  'curso': 'PPG-PIPGEs',
  'turma': 1,
  'grupo': 'Estatística',
  'carga': 1.0,
  'ementa': 'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa=104131&disciplina=EST5106',
  'nivel': 'p',
  'horarios': [],
  'id': 'EST5106,1',
  'conflitos': [],
  'noturna': False},
 {'codigo': 'EST5507',
  'nome': 'Modelos de Regressão',
  'curso': 'PPG-PIPGEs',
  'turma': 1,
  'grupo': 'Estatística',
  'carga': 1.0,
  'ementa': 'https://www.icmc.usp.br/pos-graduacao/disciplinas?programa=104131&disciplina=EST5507',
  'nivel': 'p',
  'horarios': [],
  'id': 'EST5507,1',
  'conflitos': [],
  'noturna': Fals

In [28]:
# with open(r'./app/context/dados.ts', 'w', encoding='utf-8') as f:
#     f.write('import { TurmaDataInicial } from "../types";\n')
#     f.write('\nexport const turmasJson: TurmaDataInicial[] = ')
#     json.dump(json_data, f, ensure_ascii=False, indent=2)